This notebook generates an adjacency matrix for stations or clusters.

In [1]:
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from functools import reduce
from geopy import Point
from geopy import distance

import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
EXPORT_DIR = '../data/exports/adjacency_matrix'
CENTROIDS_DIR = '../data/exports'
LABELS_DIR = '../data/exports'
SHAPEFILE_DIR = '../data/shapefiles/zipcodes'
STATIONS_DIR = '../data/exports'
TRIPS_DIR = '../data/raw'

# TODO(cpcarey): Convert to enum.
# Options include: 'displacement', 'elevation', 'trip_count',
# 'trip_count_classic', 'trip_count_electric'
VARIABLE = 'elevation'

TRIP_DATES = [
    '202007',
    '202008',
    '202009',
    '202010',
    '202011',
    '202012',
    '202101',
    '202102',
]

In [3]:
CLUSTER = True
K = 25
NODE_TYPE = 'label' if CLUSTER else 'station_id'
ID1 = f'start_{NODE_TYPE}'
ID2 = f'end_{NODE_TYPE}'

if CLUSTER:
    EXPORT_DIR = f'{EXPORT_DIR}/k{K}'

In [4]:
class AnalysisConfig:

    def __init__(self,
                 centroids_path='',
                 export_path='',
                 labels_path='',
                 stations_path='',
                 trips_path_suffix=''):
        self.centroids_path = centroids_path
        self.export_path = export_path
        self.labels_path = labels_path
        self.stations_path = stations_path
        self.trips_path_suffix = trips_path_suffix
        self.station_ids = None

    def get_station_ids(self):
        # Cache value after calculation.
        if self.station_ids == None:
            self.station_ids = set(
                pd.read_csv(self.stations_path)['station_id'].astype(str))
        return self.station_ids

    def get_trips_dfs(self):
        trips_paths = [
            '{}/{}{}'.format(TRIPS_DIR, date, self.trips_path_suffix)
            for date in TRIP_DATES
        ]
        dfs = [pd.read_csv(path) for path in trips_paths]
        for df in dfs:
            df['start_station_id'] = df['start_station_id'].astype(str)
            df['end_station_id'] = df['end_station_id'].astype(str)
        return dfs

In [5]:
config_sf = AnalysisConfig(
    centroids_path=f'{CENTROIDS_DIR}/centroids_k{K}_sf.csv',
    export_path=f'{EXPORT_DIR}/{VARIABLE}_sf.csv',
    labels_path=f'{LABELS_DIR}/cluster_labels_k{K}_sf.csv',
    stations_path=f'{STATIONS_DIR}/SF_ele_single station.csv',
    trips_path_suffix='-baywheels-tripdata.csv',
)

config_dc = AnalysisConfig(
    centroids_path=f'{CENTROIDS_DIR}/centroids_k{K}_dc.csv',
    export_path=f'{EXPORT_DIR}/{VARIABLE}_dc.csv',
    labels_path=f'{LABELS_DIR}/cluster_labels_k{K}_dc.csv',
    stations_path=f'{STATIONS_DIR}/DC_ele_single station.csv',
    trips_path_suffix='-capitalbikeshare-tripdata.csv',
)

In [6]:
config = config_dc

In [7]:
def clean_trips(df, config):
    """Drops missing and non-matching station IDs."""
    REQUIRED_COLUMNS = ['start_station_id', 'end_station_id']
    
    # Drop missing station IDs.
    new_df = df.dropna(subset=REQUIRED_COLUMNS)
    
    # Drop non-matching station IDs.
    for column in REQUIRED_COLUMNS:
        new_df = new_df[new_df[column].isin(config.get_station_ids())]
    return new_df

In [8]:
if 'trip_count' in VARIABLE:
    trips_dfs = [clean_trips(df, config) for df in config.get_trips_dfs()]
    all_trips_df = pd.concat(trips_dfs, ignore_index=True)

In [9]:
if 'trip_count' in VARIABLE:
    grouping_df = all_trips_df
    if 'classic' in VARIABLE:
        # WARNING: SF changes 'docked_bike' to 'classic_bike' over time period.
        grouping_df = grouping_df[grouping_df['rideable_type'].isin(['classic_bike', 'docked_bike'])]
    if 'electric' in VARIABLE:
        grouping_df = grouping_df[grouping_df['rideable_type'] == 'electric_bike']
    
    all_trips_counts = grouping_df.groupby(['start_station_id',
                                             'end_station_id']).agg({
                                                 'ride_id': 'count'
                                             }).rename(columns={
                                                 'ride_id': 'trip_count',
                                             }).reset_index()
        
if 'trip_count' in VARIABLE:
    display(all_trips_counts)

In [10]:
if 'trip_count' in VARIABLE:
    if CLUSTER:
        start_df = pd.read_csv(config.labels_path).rename(columns={
            'station_id': 'start_station_id',
            'label': 'start_label',
        })
        start_df['start_station_id'] = start_df['start_station_id'].astype(str)
        end_df = pd.read_csv(config.labels_path).rename(columns={
            'station_id': 'end_station_id',
            'label': 'end_label',
        })
        end_df['end_station_id'] = end_df['end_station_id'].astype(str)
        cluster_counts_df = pd.merge(all_trips_counts,
                                     start_df,
                                     on='start_station_id',
                                     how='left')
        cluster_counts_df = pd.merge(cluster_counts_df,
                                     end_df,
                                     on='end_station_id',
                                     how='left')
        cluster_counts_df = cluster_counts_df.drop(
            columns=['start_station_id', 'end_station_id'])
        cluster_counts_df = cluster_counts_df.groupby(['start_label',
                                                       'end_label']).agg({
                                                           'trip_count': 'sum'
                                                       }).reset_index()
        display(cluster_counts_df)

In [11]:
centroids_df = None
if CLUSTER:
    centroids_df = pd.read_csv(config.centroids_path)
    display(centroids_df.head())

,lat,lng,elevation,count
0,38.898156,-77.022912,11.321429,28
1,38.966477,-77.025898,79.500000,6
2,38.869956,-76.980301,11.727273,11
3,38.928787,-77.032849,56.304348,23
4,38.909630,-77.034458,25.500000,26


In [12]:
nodes_df = centroids_df if CLUSTER else stations_df

def get_distance(point1, point2):
    return distance.geodesic(point1, point2).m

def get_point(node_id):
    return Point(nodes_df.loc[node_id]['lat'], nodes_df.loc[node_id]['lng'])

def get_displacement(node_id1, node_id2):
    return get_distance(get_point(node_id1), get_point(node_id2))

def get_elevation_change(node_id1, node_id2):
    return (nodes_df.loc[node_id2]['elevation'] - 
            nodes_df.loc[node_id1]['elevation'])

def get_gradient(node_id1, node_id2):
    return get_elevation_change(node_id1, node_id2) / get_displacement(node_id1, node_id2)

def get_trip_count(node_id1, node_id2):
    df = cluster_counts_df if CLUSTER else all_trips_counts
    NODE_ID = 'label' if CLUSTER else 'station_id'
    
    row = df[(df[f'start_{NODE_ID}'] == node_id1) &
             (df[f'end_{NODE_ID}'] == node_id2)]
    if len(row) == 0:
        return 0
    return row.iloc[:, -1:].values[0][0]

In [13]:
adj_matrix = pd.DataFrame(index=nodes_df.index, columns=nodes_df.index)

if VARIABLE == 'displacement':
    adj_matrix = adj_matrix.apply(lambda row: row.index.to_series().apply(
        lambda col_name: get_displacement(row.name, col_name)),
                                  axis=1)
elif VARIABLE == 'elevation':
    adj_matrix = adj_matrix.apply(lambda row: row.index.to_series().apply(
        lambda col_name: get_elevation_change(row.name, col_name)),
                                  axis=1)
elif 'trip_count' in VARIABLE:
    adj_matrix = adj_matrix.apply(lambda row: row.index.to_series().apply(
        lambda col_name: get_trip_count(row.name, col_name)), axis=1)
    
adj_matrix.head()

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,0.000000,68.178571,0.405844,44.982919,14.178571,11.011905,18.928571,13.269481,-5.188095,106.250000,...,4.345238,-0.876984,-5.380252,6.540640,15.678571,64.964286,-4.144958,72.178571,88.678571,25.951299
1,-68.178571,0.000000,-67.772727,-23.195652,-54.000000,-57.166667,-49.250000,-54.909091,-73.366667,38.071429,...,-63.833333,-69.055556,-73.558824,-61.637931,-52.500000,-3.214286,-72.323529,4.000000,20.500000,-42.227273
2,-0.405844,67.772727,0.000000,44.577075,13.772727,10.606061,18.522727,12.863636,-5.593939,105.844156,...,3.939394,-1.282828,-5.786096,6.134796,15.272727,64.558442,-4.550802,71.772727,88.272727,25.545455
3,-44.982919,23.195652,-44.577075,0.000000,-30.804348,-33.971014,-26.054348,-31.713439,-50.171014,61.267081,...,-40.637681,-45.859903,-50.363171,-38.442279,-29.304348,19.981366,-49.127877,27.195652,43.695652,-19.031621
4,-14.178571,54.000000,-13.772727,30.804348,0.000000,-3.166667,4.750000,-0.909091,-19.366667,92.071429,...,-9.833333,-15.055556,-19.558824,-7.637931,1.500000,50.785714,-18.323529,58.000000,74.500000,11.772727


In [14]:
adj_matrix.index = adj_matrix.index.rename(NODE_TYPE)
adj_matrix.to_csv(config.export_path)

In [15]:
csv_adj_matrix = pd.read_csv(config.export_path, index_col=1)
display(csv_adj_matrix.head())

,label,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,,,,,,,,,,,,,,,,,,,,,
0.000000,0,68.178571,0.405844,44.982919,14.178571,11.011905,18.928571,13.269481,-5.188095,106.250000,...,4.345238,-0.876984,-5.380252,6.540640,15.678571,64.964286,-4.144958,72.178571,88.678571,25.951299
-68.178571,1,0.000000,-67.772727,-23.195652,-54.000000,-57.166667,-49.250000,-54.909091,-73.366667,38.071429,...,-63.833333,-69.055556,-73.558824,-61.637931,-52.500000,-3.214286,-72.323529,4.000000,20.500000,-42.227273
-0.405844,2,67.772727,0.000000,44.577075,13.772727,10.606061,18.522727,12.863636,-5.593939,105.844156,...,3.939394,-1.282828,-5.786096,6.134796,15.272727,64.558442,-4.550802,71.772727,88.272727,25.545455
-44.982919,3,23.195652,-44.577075,0.000000,-30.804348,-33.971014,-26.054348,-31.713439,-50.171014,61.267081,...,-40.637681,-45.859903,-50.363171,-38.442279,-29.304348,19.981366,-49.127877,27.195652,43.695652,-19.031621
-14.178571,4,54.000000,-13.772727,30.804348,0.000000,-3.166667,4.750000,-0.909091,-19.366667,92.071429,...,-9.833333,-15.055556,-19.558824,-7.637931,1.500000,50.785714,-18.323529,58.000000,74.500000,11.772727
